In [1]:
import tensorflow as tf
print(tf.__version__)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

2.6.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import numpy as np

# x1 = np.load('dataset_nyuv2/x1.npy',mmap_mode='r')
# x2 = np.load('dataset_nyuv2/x2.npy',mmap_mode='r')
# y = np.load('dataset_nyuv2/y.npy',mmap_mode='r')
# y = np.load('dataset_nyuv2/y_prepared_32.npy',mmap_mode='r')
# print(y.shape)
# y = np.reshape(y, (1449, 480, 640, 33), order='F')

x1 = np.load('dataset_sunrgbd/x1.npy',mmap_mode='r')
x2 = np.load('dataset_sunrgbd/x2_3.npy',mmap_mode='r')
y = np.load('dataset_sunrgbd/y_prepared.npy',mmap_mode='r')

In [3]:
from sklearn.model_selection import train_test_split

# dataset = tf.data.Dataset.from_tensor_slices([x1, x2])

x = np.array(list(zip(x1, x2)))

X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

# X1_train, X2_train = X_train[:, 0], X_train[:, 1]

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)

In [4]:
train_datagen.fit(X_train[:, 0])

train = train_datagen.flow([X_train[:, 0], X_train[:, 1]], y_train, batch_size=16, subset='training')
validation = train_datagen.flow([X_train[:, 0], X_train[:, 1]], y_train, batch_size=8, subset='validation')

NameError: name 'train_datagen' is not defined

In [7]:
from segmentation_models import Unet
from segmentation_models.utils import set_trainable
import segmentation_models as sm

sm.set_framework('tf.keras')
sm.framework()

pretrained_model_1 = Unet(backbone_name='resnet34', encoder_weights='imagenet', encoder_freeze=True, classes=66, activation='softmax', input_shape=(480, 640, 3))
pretrained_model_1.summary()
for layer in pretrained_model_1.layers:
    layer._name += '_1'

Segmentation Models: using `keras` framework.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, 480, 640, 3) 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 480, 640, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 486, 646, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 240, 320, 64) 9408        zero_padding2d[0][0]             
______________________________________________

In [8]:
pretrained_model_2 = Unet(backbone_name='resnet34', encoder_weights='imagenet', encoder_freeze=True, classes=66, activation='softmax', input_shape=(480, 640, 3))
pretrained_model_2.summary()
for layer in pretrained_model_2.layers:
    layer._name += '_2'

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, 480, 640, 3) 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 480, 640, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_34 (ZeroPadding2 (None, 486, 646, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 240, 320, 64) 9408        zero_padding2d_34[0][0]          
____________________________________________________________________________________________

In [9]:
#last layers
rgb_last = pretrained_model_1.get_layer('add_15_1')
depth_last = pretrained_model_2.get_layer('add_31_2')
print('model_1 layer output shape: ', rgb_last.output_shape)
print('model_2 layer output shape: ', depth_last.output_shape)
output_1 = rgb_last.output
output_2 = depth_last.output

# Middle layers
rgb_stage_1 = pretrained_model_1.get_layer('stage1_unit1_relu1_1')
depth_stage_1 = pretrained_model_2.get_layer('stage1_unit1_relu1_2')
rgb_stage_2 = pretrained_model_1.get_layer('stage2_unit1_relu1_1')
depth_stage_2 = pretrained_model_2.get_layer('stage2_unit1_relu1_2')
rgb_stage_3 = pretrained_model_1.get_layer('stage3_unit1_relu1_1')
depth_stage_3 = pretrained_model_2.get_layer('stage3_unit1_relu1_2')
rgb_stage_4 = pretrained_model_1.get_layer('stage4_unit1_relu1_1')
depth_stage_4 = pretrained_model_2.get_layer('stage4_unit1_relu1_2')

rgb_fusion_1 = rgb_stage_1.output
depth_fusion_1 = depth_stage_1.output
rgb_fusion_2 = rgb_stage_2.output
depth_fusion_2 = depth_stage_2.output
rgb_fusion_3 = rgb_stage_3.output
depth_fusion_3 = depth_stage_3.output
rgb_fusion_4 = rgb_stage_4.output
depth_fusion_4 = depth_stage_4.output

model_1 layer output shape:  (None, 15, 20, 512)
model_2 layer output shape:  (None, 15, 20, 512)


In [10]:
def rgbd_fusion(input_r,input_d,reshape_size,label):
    r = tf.keras.layers.GlobalAveragePooling2D()(input_r)
    r = tf.keras.layers.Reshape((1,1,reshape_size))(r)
    r = tf.keras.layers.Conv2D(32,kernel_size=(1,1),strides=(1,1),activation='relu')(r)
    r = tf.keras.layers.Conv2D(reshape_size,kernel_size=(1,1),strides=(1,1),activation='sigmoid')(r)
    m_1 = tf.keras.layers.Multiply()([input_r,r])
    
    d = tf.keras.layers.GlobalAveragePooling2D()(input_d)
    d = tf.keras.layers.Reshape((1,1,reshape_size))(d)
    d = tf.keras.layers.Conv2D(32,kernel_size=(1,1),strides=(1,1),activation='relu')(d)
    d = tf.keras.layers.Conv2D(reshape_size,kernel_size=(1,1),strides=(1,1),activation='sigmoid')(d)
    m_2 = tf.keras.layers.Multiply()([input_d,d])
    name = 'fusion' + label
    last = tf.keras.layers.Add(name=name)([m_1,m_2])
    
    return last

def decode_layer(input_de,input_ad,filters):
    x = tf.keras.layers.UpSampling2D(size=(2,2), interpolation="nearest")(input_de)
    x = tf.keras.layers.Concatenate()([x,input_ad])
    x = tf.keras.layers.Conv2D(filters,kernel_size=(3,3),activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = tf.keras.layers.Conv2D(filters,kernel_size=(3,3),activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    return x


def last_decode_layer(input_de,filters):
    x = tf.keras.layers.UpSampling2D(size=(2,2), interpolation="nearest")(input_de)
    x = tf.keras.layers.Conv2D(filters,kernel_size=(3,3),activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = tf.keras.layers.Conv2D(filters,kernel_size=(3,3),activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    return x

In [11]:
fusion_1 = rgbd_fusion(rgb_fusion_1,depth_fusion_1,64,'_1')
print(fusion_1.shape)
fusion_2 = rgbd_fusion(rgb_fusion_2,depth_fusion_2,64,'_2')
print(fusion_2.shape)
fusion_3 = rgbd_fusion(rgb_fusion_3,depth_fusion_3,128,'_3')
print(fusion_3.shape)
fusion_4 = rgbd_fusion(rgb_fusion_4,depth_fusion_4,256,'_4')
print(fusion_4.shape)
fusion_last = rgbd_fusion(output_1, output_2,512,'_last')
print(fusion_last.shape)

(None, 120, 160, 64)
(None, 120, 160, 64)
(None, 60, 80, 128)
(None, 30, 40, 256)
(None, 15, 20, 512)


In [16]:
xr = decode_layer(fusion_last,fusion_4,1024)
xr = decode_layer(xr,fusion_3,512)
xr = decode_layer(xr,fusion_2,256)
xr = decode_layer(xr,tf.keras.layers.UpSampling2D(size=(2,2), interpolation="nearest")(fusion_1),128)
xr = last_decode_layer(xr,64)

xr = tf.keras.layers.Conv2D(47, kernel_size=(3, 3), activation='relu', padding='same')(xr)
xr = tf.keras.layers.Conv2D(47, kernel_size=(1, 1), activation='relu', padding='same')(xr)
xr = tf.keras.layers.Conv2D(47, kernel_size=(1, 1), activation='softmax', padding='same', name='last_layer')(xr)

model = tf.keras.Model(inputs=[pretrained_model_1.input, pretrained_model_2.input], outputs=xr, name='myModel')
# encoder_fusion = tf.keras.Model(inputs=[pretrained_model_1.input, pretrained_model_2.input], outputs=fusion)
model.summary()

Model: "myModel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data_1 (InputLayer)             [(None, 480, 640, 3) 0                                            
__________________________________________________________________________________________________
data_2 (InputLayer)             [(None, 480, 640, 3) 0                                            
__________________________________________________________________________________________________
bn_data_1 (BatchNormalization)  (None, 480, 640, 3)  9           data_1[0][0]                     
__________________________________________________________________________________________________
bn_data_2 (BatchNormalization)  (None, 480, 640, 3)  9           data_2[0][0]                     
____________________________________________________________________________________________

In [17]:
stage_fusion = model.get_layer('last_layer')
print(' layer output shape: ', stage_fusion.output_shape)

 layer output shape:  (None, 480, 640, 47)


In [14]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_best_weights.hdf5".format('seg_model')

checkpoint = ModelCheckpoint(weight_path, monitor='val_dice_coef', verbose=1, 
                             save_best_only=True, mode='max', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_dice_coef', factor=0.5, 
                                   patience=3, 
                                   verbose=1, mode='max', min_delta=0.0001, cooldown=2, min_lr=1e-6)
early = EarlyStopping(monitor="val_dice_coef", 
                      mode="max", 
                      patience=50) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [11]:
model.compile(loss="binary_crossentropy", optimizer= "Adam", metrics =["accuracy"])

# Para utilizar el 20% para validación vamos a poner que realizen un split de 0,2
history = model.fit(train, epochs=30, verbose=1,
                    batch_size=8, validation_data=validation, callbacks=callbacks_list)

Epoch 1/30
145/145 [==============================] - 33s 189ms/step - loss: 0.0440 - accuracy: 0.0950 - val_loss: 0.0463 - val_accuracy: 0.0998
Epoch 2/30
145/145 [==============================] - 21s 146ms/step - loss: 0.0435 - accuracy: 0.0982 - val_loss: 0.0454 - val_accuracy: 0.1021
Epoch 3/30
145/145 [==============================] - 22s 151ms/step - loss: 0.0429 - accuracy: 0.1025 - val_loss: 0.0456 - val_accuracy: 0.0958
Epoch 4/30
145/145 [==============================] - 21s 147ms/step - loss: 0.0425 - accuracy: 0.1044 - val_loss: 0.0445 - val_accuracy: 0.1088
Epoch 5/30
145/145 [==============================] - 22s 149ms/step - loss: 0.0422 - accuracy: 0.1063 - val_loss: 0.0441 - val_accuracy: 0.1075
Epoch 6/30
145/145 [==============================] - 22s 149ms/step - loss: 0.0417 - accuracy: 0.1091 - val_loss: 0.0445 - val_accuracy: 0.1078
Epoch 7/30
145/145 [==============================] - 22s 155ms/step - loss: 0.0414 - accuracy: 0.1106 - val_loss: 0.0449 - val_ac